In [1]:
import pandas as pd
import numpy as np

from pathlib import Path
import time

pd.options.mode.chained_assignment = None  # default='warn'

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
## https://developers.refinitiv.com/article/find-your-right-companies-screener-eikon-data-apispython
# https://www.reddit.com/r/webdev/comments/t2s836/what_is_the_easiest_way_to_hide_your_api_key_on/
import eikon as ek
ek.set_app_key('341ab68907a740f38e7b07084d4055e3982b5bdf')

# M&A screening

<pre>•Deals zwischen Juni 2008 und September 2022
•Deal status: Completed & withdrawn
•Nation classification: US,Canada,Australia, United Kingdom
•Deal value: größer 1Mio USD
•M&A Type: Disclosed dollar value deal
•% of shares before deal kleiner 50%, danach größer 50%
•Company status: public</pre>

## retrieve sample

In [5]:
'''
unbedingt "&" removen aus der excel funktion. den rest kann man anscheinend direkt übernehmen
'''

firms = 'SCREEN(U(IN(DEALS)),' + ' \
            BETWEEN(TR.MnAAnnDate,20080107,20213112), \
            IN(TR.MnAAcquirorNation,"AU","GB","CA","US"), \
            IN(TR.MnATargetNation,"AU","GB","CA","US"), \
            IN(TR.MnAStatus,"W","SW","IW","DR","U","C"), \
            TR.MnADealValue(Scale=6)>=1, \
            IN(TR.MnADealType,"DI"), \
            TR.MnAPctOfSharesHeld6MonthsPrior<50, \
            TR.MnAPctOfSharesOwnedPostMerger>50, \
            IN(TR.MnAAcquirorPublicStatus,"P", "S"), \
            IN(TR.MnATargetPublicStatus,"P", "S"), \
            CURN=USD) \
            '
            
fields = ['TR.MnASDCDealNumber,TR.MnAAnnDate,TR.MnADateUnconditional,TR.MnAEffectiveDate,TR.MnAStatus,TR.MnATarget,TR.MnATargetPermId,TR.MnATargetSDCCusip,TR.MnATargetNation,TR.MnATargetPublicStatus,TR.MnATargetTotalAssetsLTM,TR.MnAAcquiror,TR.MnAAcquirorPermId,TR.MnAAcquirorSDCCusip,TR.MnAAcquirorSEDOL,TR.MnAAcquirorNation,TR.MnAAcquirorPublicStatus,TR.MnAAcquirorTotalAssetsLTM, TR.MnADealValue(Curn=USD,Scale=6),TR.MnAPctOfSharesHeld6MonthsPrior,TR.MnAPctOfSharesOwnedPostMerger'] #can be placed in just one string

df_ma, err = ek.get_data(firms, fields)

df_ma.head()
df_ma.shape

'\nunbedingt "&" removen aus der excel funktion. den rest kann man anscheinend direkt übernehmen\n            BETWEEN(TR.MnAAnnDate,20210112,20211512),             BETWEEN(TR.MnAAnnDate,20080107,20213112),             IN(TR.MnAAcquirorPublicStatus,"P"), \\ #also include subsidiaries (most of them are irrelevant though)\n'

,Instrument,SDC Deal No,Date Announced,Date Effective or Unconditional,Date Effective,Deal Status,Target Full Name,Target PermID,Target 6-digit CUSIP,Target Nation,...,Acquiror Full Name,Acquiror PermID,Acquiror 6-digit CUSIP,Acquiror's SEDOL,Acquiror Nation,Acquiror Public Status,Acquiror Total Assets Last 12 Months,Deal Value,Percentage of Shares Held by Acquiror 6 Months Prior to Announcement,Percentage of Shares Owned after Transaction
0,34391097466,2116013020,2009-10-07,2009-12-08,2009-12-08,Completed,Trinity Alps Resources Inc- Discovery Day Gold...,<NA>,25581K,United States,...,American Sierra Gold Corp,4298488815.0,029616,B2NS4Z7,United States,Public,<NA>,3.7,0.0,75.0
1,34391276852,1761253020,2011-03-28,2011-05-02,2011-05-02,Completed,Fulton Street Brewery LLC,5035166382.0,14681L,United States,...,Anheuser Busch Inc,4296453080.0,035231,<NA>,United States,Subsidiary,<NA>,22.5,0.0,58.0
2,34391279156,1780964020,2010-05-21,2010-06-29,2010-06-29,Completed,Michael Foods Inc,4295907158.0,594079,United States,...,GS Capital Partners LP,5000878558.0,36224Z,<NA>,United States,Subsidiary,<NA>,1700.0,0.0,100.0
3,34391281082,1794406020,2008-09-15,2008-10-30,2008-10-30,Completed,Napster Inc,4295899555.0,630797,United States,...,Best Buy Co Inc,4295903556.0,086516,2094670,United States,Public,17522000000.0,126.935,0.0,90.191
4,34391290134,1879131020,2008-04-24,2008-09-29,2008-09-29,Completed,Wendy's International Inc,4295905330.0,950590,United States,...,Triarc Cos Inc,4295905130.0,895927,B3NXMJ9,United States,Public,1390735000.0,2345.517,0.0,100.0


(15574, 22)

In [99]:
## save file with dynamic file name
dir = Path("W:/019_Glassdoor/1 Data/1 Glassdoor Links/MnA Sample Eikon")
path_with_time = Path.joinpath(dir, f'{time.strftime("%m%d")}_MnA_Sample_Eikon_unfiltered.csv')

df_ma.to_csv(path_with_time, sep=";", decimal=",", float_format='%.2f', index=False) #german decimal

print(f"{path_with_time} saved")

W:\019_Glassdoor\1 Data\1 Glassdoor Links\MnA Sample Eikon\0920_MnA_Sample_Eikon_unfiltered.csv saved


### prepare sample

In [26]:
df = df_ma.copy()

df = df.rename(columns={"Instrument": "eikon_deal_id", "Total Assets Last 12 Months": "Target Total Assets Last 12 Months"}) #rename
df['Date Announced'] = pd.to_datetime(df['Date Announced']) #convert date strings to date objects

# filter df
condition = (
    (df["Target Public Status"] == "Public") #exclude subsidiary
    & (df["Acquiror Public Status"] == "Public") 
    & (~df['Date Announced'].dt.year.isin([])) #exclude 2008, 2009, 2010
    # & (df["Target PermID"].isna() == True) #exclude empty PermID //some of them have financials -> keep them
    # & (df["Target 6-digit CUSIP"].isna() == True) # none missing
    # & (df['Total Assets Last 12 Months'] < df['Total Assets Last 12 Months'].quantile(0.1)) # < : exclude all above quantile
    )
df = df[condition]

df.head()
df.shape

missing_isins = df[df["Target PermID"].isna() == True].shape[0]
print(f"Obs with missing target ISIN: {missing_isins}")

,eikon_deal_id,SDC Deal No,Date Announced,Date Effective or Unconditional,Date Effective,Deal Status,Target Full Name,Target PermID,Target 6-digit CUSIP,Target Nation,...,Acquiror Full Name,Acquiror PermID,Acquiror 6-digit CUSIP,Acquiror's SEDOL,Acquiror Nation,Acquiror Public Status,Acquiror Total Assets Last 12 Months,Deal Value,Percentage of Shares Held by Acquiror 6 Months Prior to Announcement,Percentage of Shares Owned after Transaction
3,34391281082,1794406020,2008-09-15,2008-10-30,2008-10-30,Completed,Napster Inc,4295899555.0,630797,United States,...,Best Buy Co Inc,4295903556.0,086516,2094670,United States,Public,17522000000.0,126.935,0.0,90.191
4,34391290134,1879131020,2008-04-24,2008-09-29,2008-09-29,Completed,Wendy's International Inc,4295905330.0,950590,United States,...,Triarc Cos Inc,4295905130.0,895927,B3NXMJ9,United States,Public,1390735000.0,2345.517,0.0,100.0
8,34391292476,1900306020,2008-01-28,2008-08-22,2008-08-22,Completed,NYMEX Holdings Inc,8589934166.0,62948N,United States,...,CME Group Inc,4295899615.0,12572Q,2965839,United States,Public,18562632000.0,7555.372,0.0,100.0
14,34391297274,1941059020,2008-01-07,2008-04-01,2008-04-01,Completed,Burke-Parsons-Bowlby Corp,4296470998.0,121368,United States,...,Stella-Jones Inc,5048817617.0,85853F,2809777,Canada,Public,263939000.0,76.132,0.0,100.0
17,34391297298,1941224020,2008-01-08,2008-07-18,2008-07-18,Completed,"MFB Corp,Mishawaka,Indiana",4295907031.0,55272D,United States,...,MutualFirst Financial Inc,4295914259.0,62845B,2620343,United States,Public,957343000.0,57.168,0.0,100.0


(3142, 22)

Obs with missing target ISIN: 32


## cusip to ISIN (totally unreliable, use permid to isin instead)

In [ ]:
firms_cusips = df["Target 6-digit CUSIP"].tolist()

df_cusips = ek.get_symbology(firms_cusips, from_symbol_type='CUSIP', to_symbol_type=None, raw_output=False, debug=False)
# to_symbol_type=None -> all are requested. CUSIP , ISIN , SEDOL , RIC , ticker , lipperID , IMO , and OAPermID

df_cusips = df_cusips[df_cusips["ISIN"].isna() == False]
df_cusips

## permid to isin

#### target

In [7]:
# create target permid list
target_cusips = df["Target PermID"].astype("Int64").astype(str).tolist() #first convert to int (without decimal), then to str, since get_data needs strings in the list
fields = ['TR.ISIN', 'TR.CUSIP', 'TR.RIC'] # check the data item browser
params = {'Scale': 6, 'SDate': 0, 'EDate': 0, 'FRQ': 'FY', 'Curn': 'USD'} # set EDate to minus values for the most recent years

df_isin_target_retrieve, err = ek.get_data(target_cusips, fields, params)

df_isin_target_retrieve

,Instrument,ISIN,CUSIP,RIC
0,4295899555,US6307971084,630797108,NAPS.OQ^K08
1,4295905330,US9505901093,950590109,WEN.N^I08
2,8589934166,US62948N1046,62948N104,NMX.N^H08
3,4296470998,US1213681042,121368104,BPAB.PK^E08
4,4295907031,US55272D1063,55272D106,MFBC.OQ^G08
...,...,...,...,...
3137,5064695004,CA72941N1006,72941N100,PLUS.CD^I21
3138,5000095185,AU000000WIC6,,WIC.AX
3139,4295856861,AU000000OZG8,,OZG.AX
3140,5079547980,GB00BMDNH979,,OCDX.OQ


In [8]:
df_isin_target = df_isin_target_retrieve.copy()
#replace empty strings with np nan
for columnname in ['ISIN', 'CUSIP', 'RIC']:
  df_isin_target[columnname] = df_isin_target[columnname].replace({"":np.nan})

df_isin_target = df_isin_target.rename(columns={"Instrument": "Target PermID"})

# add prefix to columns (except first column)
new_names = [(i,'Target_'+i) for i in df_isin_target.iloc[:, 1:].columns.values] #set up dict entries for each columns name {"old": "Target_old"}
df_isin_target.rename(columns = dict(new_names), inplace=True)

# only display rows in which isin is not missing
df_isin_target = df_isin_target[df_isin_target["Target_ISIN"].isna() == False]
df_isin_target

,Target PermID,Target_ISIN,Target_CUSIP,Target_RIC
0,4295899555,US6307971084,630797108,NAPS.OQ^K08
1,4295905330,US9505901093,950590109,WEN.N^I08
2,8589934166,US62948N1046,62948N104,NMX.N^H08
3,4296470998,US1213681042,121368104,BPAB.PK^E08
4,4295907031,US55272D1063,55272D106,MFBC.OQ^G08
...,...,...,...,...
3137,5064695004,CA72941N1006,72941N100,PLUS.CD^I21
3138,5000095185,AU000000WIC6,<NA>,WIC.AX
3139,4295856861,AU000000OZG8,<NA>,OZG.AX
3140,5079547980,GB00BMDNH979,<NA>,OCDX.OQ


#### acq

In [9]:
acquiror_cusips = df["Acquiror PermID"].astype("Int64").astype(str).tolist() #first convert to int (without decimal), then to str, since get_data needs strings in the list
fields = ['TR.ISIN', 'TR.CUSIP', 'TR.RIC'] # check the data item browser
params = {'Scale': 6, 'SDate': 0, 'EDate': 0, 'FRQ': 'FY', 'Curn': 'USD'} # set EDate to minus values for the most recent years

df_isin_acquiror_retrieve, err = ek.get_data(acquiror_cusips, fields, params)


df_isin_acquiror = df_isin_acquiror_retrieve.copy()
#replace empty strings with np nan
for columnname in ['ISIN', 'CUSIP', 'RIC']:
  df_isin_acquiror[columnname] = df_isin_acquiror[columnname].replace({"":np.nan})

df_isin_acquiror = df_isin_acquiror.rename(columns={"Instrument": "Acquiror PermID"})

# add prefix to columns (except first column)
new_names = [(i,'Acquiror_'+i) for i in df_isin_acquiror.iloc[:, 1:].columns.values] #set up dict entries for each columns name {"old": "acquiror_old"}
df_isin_acquiror.rename(columns = dict(new_names), inplace=True)

# only display rows in which isin is not missing
df_isin_acquiror = df_isin_acquiror[df_isin_acquiror["Acquiror_ISIN"].isna() == False]
df_isin_acquiror

,Acquiror PermID,Acquiror_ISIN,Acquiror_CUSIP,Acquiror_RIC
0,4295903556,US0865161014,086516101,BBY.N
1,4295905130,US95058W1009,95058W100,WEN.OQ
2,4295899615,US12572Q1058,12572Q105,CME.OQ
3,5048817617,CA85853F1053,85853F105,SJ.TO
4,4295914259,US62845B1044,62845B104,MFSF.OQ^D20
...,...,...,...,...
3137,5068929693,CA3771304068,377130406,GLASau.NLB
3138,4295858256,AU000000WAM2,<NA>,WAM.AX
3139,4295858256,AU000000WAM2,<NA>,WAM.AX
3140,4295907711,US74838J1016,74838J101,QDEL.OQ


#### merge acq / target ISINs on PermID (df also includes obs with missing isin)

In [27]:
# perm IDs are stored as string -> convert to Int64
for columnname in ["Acquiror PermID", "Target PermID"]:
    df[columnname] = df[columnname].astype("Int64")

df_isin_acquiror["Acquiror PermID"] = df_isin_acquiror["Acquiror PermID"].astype("Int64")
df_isin_target["Target PermID"] = df_isin_target["Target PermID"].astype("Int64")

# merge acq / target ISINs on PermID
df_final = df.merge(df_isin_acquiror.iloc[:,:2], how="left", on="Acquiror PermID")
df_final = df_final.merge(df_isin_target.iloc[:,:2], how="left", on="Target PermID")

# there are duplicates since multiple acquiror possible
df_final = (df_final.drop_duplicates()
            .reset_index(drop=True))

# create identifier based on acq + target isin
df_final["acq_target_isin"] = df_final["Acquiror_ISIN"] + df_final["Target_ISIN"]

# shift columns (based on dict)
columns_to_move = {"Acquiror_ISIN": "Acquiror PermID", "Target_ISIN": "Target PermID", "acq_target_isin": "SDC Deal No"}
for key, value in columns_to_move.items():
    poped_col = df_final.pop(key)
    df_final.insert(df_final.columns.get_loc(value)+1, key, poped_col)

# replace whitespaces in column names with underscore & make all column names lowercase
df_final.columns = df_final.columns.str.replace(' ', '_').str.lower()

# remove obs with missing target ISIN
df_final = df_final[df_final["target_isin"].isna() == False] # ignore them. some of them have a DE isin based on the given cusip, which doesnt make sense

df_final.head()
df_final.shape

missing_isins = df_final[df_final["target_isin"].isna() == True].shape[0]
print(f"Obs with missing target ISIN: {missing_isins}")

,eikon_deal_id,sdc_deal_no,acq_target_isin,date_announced,date_effective_or_unconditional,date_effective,deal_status,target_full_name,target_permid,target_isin,...,acquiror_permid,acquiror_isin,acquiror_6-digit_cusip,acquiror's_sedol,acquiror_nation,acquiror_public_status,acquiror_total_assets_last_12_months,deal_value,percentage_of_shares_held_by_acquiror_6_months_prior_to_announcement,percentage_of_shares_owned_after_transaction
0,34391281082,1794406020,US0865161014US6307971084,2008-09-15,2008-10-30,2008-10-30,Completed,Napster Inc,4295899555,US6307971084,...,4295903556,US0865161014,086516,2094670,United States,Public,17522000000.0,126.935,0.0,90.191
1,34391290134,1879131020,US95058W1009US9505901093,2008-04-24,2008-09-29,2008-09-29,Completed,Wendy's International Inc,4295905330,US9505901093,...,4295905130,US95058W1009,895927,B3NXMJ9,United States,Public,1390735000.0,2345.517,0.0,100.0
2,34391292476,1900306020,US12572Q1058US62948N1046,2008-01-28,2008-08-22,2008-08-22,Completed,NYMEX Holdings Inc,8589934166,US62948N1046,...,4295899615,US12572Q1058,12572Q,2965839,United States,Public,18562632000.0,7555.372,0.0,100.0
3,34391297274,1941059020,CA85853F1053US1213681042,2008-01-07,2008-04-01,2008-04-01,Completed,Burke-Parsons-Bowlby Corp,4296470998,US1213681042,...,5048817617,CA85853F1053,85853F,2809777,Canada,Public,263939000.0,76.132,0.0,100.0
4,34391297298,1941224020,US62845B1044US55272D1063,2008-01-08,2008-07-18,2008-07-18,Completed,"MFB Corp,Mishawaka,Indiana",4295907031,US55272D1063,...,4295914259,US62845B1044,62845B,2620343,United States,Public,957343000.0,57.168,0.0,100.0


(2874, 25)

Obs with missing target ISIN: 0


### firm SIC

In [49]:
df_trbc = df_final.copy()

df_acquiror_trbc, df_target_trbc = pd.DataFrame(), pd.DataFrame()
target_acq_isins = {"acquiror_isin": df_acquiror_trbc, "target_isin": df_target_trbc}

# retrieve acq and target trbc codes
for isin_column, datafr in target_acq_isins.items():
    firms = df_final[isin_column].astype(str).tolist()
    fields = ['TR.TRBCIndustryGroupCode', 'TR.TRBCIndustryGroup'] # check the data item browser
    params = {'Scale': 2, 'SDate': 0, 'EDate': 0, 'FRQ': 'FY', 'Curn': 'USD'} # set EDate to minus values for the most recent years

    datafr, err = ek.get_data(firms, fields, params)

    # replace whitespaces in column names with underscore & make all column names lowercase
    datafr.columns = datafr.columns.str.replace(' ', '_').str.lower()

    datafr = datafr.rename(columns={"instrument": isin_column})
    datafr.head()
    datafr.shape

    # merge with df_trbc
    df_trbc = df_trbc.merge(datafr, how="left", on=isin_column)
    df_trbc = df_trbc.drop_duplicates()

    # shift columns (based on dict)
    columns_to_move = {list(df_trbc.columns)[-1]: isin_column, list(df_trbc.columns)[-2]: isin_column}
    for key, value in columns_to_move.items():
        poped_col = df_trbc.pop(key)
        df_trbc.insert(df_trbc.columns.get_loc(value)+3, key, poped_col)


df_trbc = df_trbc.rename(columns={"trbc_industry_group_code_x": "acquiror_trbc_industry_group_code", "trbc_industry_group_name_x": "acquiror_trbc_industry_group_name", "trbc_industry_group_code_y": "target_trbc_industry_group_code", "trbc_industry_group_name_y": "target_trbc_industry_group_name"})

# mark 551020 Investment Banking & Investment Services (n=114)
df_trbc["investment acquiror"] = np.where((df_trbc["acquiror_trbc_industry_group_code"].isna() == False) & (df_trbc["acquiror_trbc_industry_group_code"] == 551020), 1, 0)
poped_col = df_trbc.pop("investment acquiror")
df_trbc.insert(df_trbc.columns.get_loc("acquiror_trbc_industry_group_name")+1, "investment acquiror", poped_col)

df_trbc.head()
df_trbc.shape

,acquiror_isin,trbc_industry_group_code,trbc_industry_group_name
0,US0865161014,534030,Specialty Retailers
1,US95058W1009,533010,Hotels & Entertainment Services
2,US12572Q1058,551020,Investment Banking & Investment Services
3,CA85853F1053,513010,Paper & Forest Products
4,US62845B1044,551010,Banking Services


(2874, 3)

,target_isin,trbc_industry_group_code,trbc_industry_group_name
0,US6307971084,572010,Software & IT Services
1,US9505901093,533010,Hotels & Entertainment Services
2,US62948N1046,551020,Investment Banking & Investment Services
3,US1213681042,513010,Paper & Forest Products
4,US55272D1063,551010,Banking Services


(2874, 3)

,eikon_deal_id,sdc_deal_no,acq_target_isin,date_announced,date_effective_or_unconditional,date_effective,deal_status,target_full_name,target_permid,target_isin,...,acquiror's_sedol,acquiror_trbc_industry_group_code,acquiror_trbc_industry_group_name,investment acquiror,acquiror_nation,acquiror_public_status,acquiror_total_assets_last_12_months,deal_value,percentage_of_shares_held_by_acquiror_6_months_prior_to_announcement,percentage_of_shares_owned_after_transaction
0,34391281082,1794406020,US0865161014US6307971084,2008-09-15,2008-10-30,2008-10-30,Completed,Napster Inc,4295899555,US6307971084,...,2094670,534030,Specialty Retailers,0,United States,Public,17522000000.0,126.935,0.0,90.191
1,34391290134,1879131020,US95058W1009US9505901093,2008-04-24,2008-09-29,2008-09-29,Completed,Wendy's International Inc,4295905330,US9505901093,...,B3NXMJ9,533010,Hotels & Entertainment Services,0,United States,Public,1390735000.0,2345.517,0.0,100.0
2,34391292476,1900306020,US12572Q1058US62948N1046,2008-01-28,2008-08-22,2008-08-22,Completed,NYMEX Holdings Inc,8589934166,US62948N1046,...,2965839,551020,Investment Banking & Investment Services,1,United States,Public,18562632000.0,7555.372,0.0,100.0
3,34391297274,1941059020,CA85853F1053US1213681042,2008-01-07,2008-04-01,2008-04-01,Completed,Burke-Parsons-Bowlby Corp,4296470998,US1213681042,...,2809777,513010,Paper & Forest Products,0,Canada,Public,263939000.0,76.132,0.0,100.0
4,34391297298,1941224020,US62845B1044US55272D1063,2008-01-08,2008-07-18,2008-07-18,Completed,"MFB Corp,Mishawaka,Indiana",4295907031,US55272D1063,...,2620343,551010,Banking Services,0,United States,Public,957343000.0,57.168,0.0,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2897,154087420083,3850353040,CA3771304068CA72941N1006,2021-12-20,2022-04-28,2022-04-28,Completed,Plus Products Inc,5064695004,CA72941N1006,...,BMVQ9T2,<NA>,,0,United States,Public,258564000.0,25.6,0.0,100.0
2898,154087421587,3851037040,AU000000WAM2AU000000WIC6,2021-12-23,2022-04-21,2022-04-21,Completed,Westoz Investment Co Ltd,5000095185,AU000000WIC6,...,6164988,551020,Investment Banking & Investment Services,1,Australia,Public,1746310000.0,156.801224,0.0,100.0
2899,154087421802,3851153040,AU000000WAM2AU000000OZG8,2021-12-23,2022-04-21,2022-04-21,Completed,Ozgrowth Ltd,4295856861,AU000000OZG8,...,6164988,551020,Investment Banking & Investment Services,1,Australia,Public,1746310000.0,106.059947,0.0,100.0
2900,154087421847,3851185020,US74838J1016GB00BMDNH979,2021-12-23,2022-05-27,2022-05-27,Completed,Ortho Clinical Diagnostics Holdings PLC,5079547980,GB00BMDNH979,...,<NA>,<NA>,<NA>,0,United States,Public,2062388000.0,8094.559,0.0,100.0


(2874, 30)

In [53]:
## save file with dynamic file name
dir = Path("W:/019_Glassdoor/1 Data/1 Glassdoor Links/MnA Sample Eikon")
path_with_time = Path.joinpath(dir, f'{time.strftime("%m%d")}_MnA_Sample_EikonWithISINandTRBC_2874.csv')

df_trbc.to_csv(path_with_time, sep=";", decimal=",", float_format='%.3f', index=False) #english decimal

print(f"{path_with_time} saved")

W:\019_Glassdoor\1 Data\1 Glassdoor Links\MnA Sample Eikon\0926_MnA_Sample_EikonWithISINandTRBC_2874.csv saved


### create one long isin list to scrape

In [54]:
df_trbc

,eikon_deal_id,sdc_deal_no,acq_target_isin,date_announced,date_effective_or_unconditional,date_effective,deal_status,target_full_name,target_permid,target_isin,...,acquiror's_sedol,acquiror_trbc_industry_group_code,acquiror_trbc_industry_group_name,investment acquiror,acquiror_nation,acquiror_public_status,acquiror_total_assets_last_12_months,deal_value,percentage_of_shares_held_by_acquiror_6_months_prior_to_announcement,percentage_of_shares_owned_after_transaction
0,34391281082,1794406020,US0865161014US6307971084,2008-09-15,2008-10-30,2008-10-30,Completed,Napster Inc,4295899555,US6307971084,...,2094670,534030,Specialty Retailers,0,United States,Public,17522000000.0,126.935,0.0,90.191
1,34391290134,1879131020,US95058W1009US9505901093,2008-04-24,2008-09-29,2008-09-29,Completed,Wendy's International Inc,4295905330,US9505901093,...,B3NXMJ9,533010,Hotels & Entertainment Services,0,United States,Public,1390735000.0,2345.517,0.0,100.0
2,34391292476,1900306020,US12572Q1058US62948N1046,2008-01-28,2008-08-22,2008-08-22,Completed,NYMEX Holdings Inc,8589934166,US62948N1046,...,2965839,551020,Investment Banking & Investment Services,1,United States,Public,18562632000.0,7555.372,0.0,100.0
3,34391297274,1941059020,CA85853F1053US1213681042,2008-01-07,2008-04-01,2008-04-01,Completed,Burke-Parsons-Bowlby Corp,4296470998,US1213681042,...,2809777,513010,Paper & Forest Products,0,Canada,Public,263939000.0,76.132,0.0,100.0
4,34391297298,1941224020,US62845B1044US55272D1063,2008-01-08,2008-07-18,2008-07-18,Completed,"MFB Corp,Mishawaka,Indiana",4295907031,US55272D1063,...,2620343,551010,Banking Services,0,United States,Public,957343000.0,57.168,0.0,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2897,154087420083,3850353040,CA3771304068CA72941N1006,2021-12-20,2022-04-28,2022-04-28,Completed,Plus Products Inc,5064695004,CA72941N1006,...,BMVQ9T2,<NA>,,0,United States,Public,258564000.0,25.6,0.0,100.0
2898,154087421587,3851037040,AU000000WAM2AU000000WIC6,2021-12-23,2022-04-21,2022-04-21,Completed,Westoz Investment Co Ltd,5000095185,AU000000WIC6,...,6164988,551020,Investment Banking & Investment Services,1,Australia,Public,1746310000.0,156.801224,0.0,100.0
2899,154087421802,3851153040,AU000000WAM2AU000000OZG8,2021-12-23,2022-04-21,2022-04-21,Completed,Ozgrowth Ltd,4295856861,AU000000OZG8,...,6164988,551020,Investment Banking & Investment Services,1,Australia,Public,1746310000.0,106.059947,0.0,100.0
2900,154087421847,3851185020,US74838J1016GB00BMDNH979,2021-12-23,2022-05-27,2022-05-27,Completed,Ortho Clinical Diagnostics Holdings PLC,5079547980,GB00BMDNH979,...,<NA>,<NA>,<NA>,0,United States,Public,2062388000.0,8094.559,0.0,100.0


# examples

## get panel data

### firm fundamentals

In [31]:
firms = ['5030853586', '4295906579'] # check the screener
fields = ['TR.Revenue.date','TR.Revenue','TR.GrossProfit', 'TR.ISIN', 'TR.RIC', 'TR.CommonName'] # check the data item browser
params = {'Scale': 2, 'SDate': -15, 'EDate': -10, 'FRQ': 'FY', 'Curn': 'USD'} # set EDate to minus values for the most recent years

df, err = ek.get_data(firms, fields, params)

df

,Instrument,Date,Revenue,Gross Profit,ISIN,RIC,Company Common Name
0,5030853586,2006-12-31T00:00:00Z,80365121.5177,48347444.409,US38259P5089,GOOG.OQ,Alphabet Inc
1,5030853586,2007-12-31T00:00:00Z,113743477.037,68167323.9045,,,
2,5030853586,2008-12-31T00:00:00Z,155927544.255,94248428.1804,,,
3,5030853586,2009-12-31T00:00:00Z,165203912.6676,103426000.5696,,,
4,5030853586,2010-12-31T00:00:00Z,219189135.5,141316852.0,,,
5,5030853586,2011-12-31T00:00:00Z,292816125.0,190938825.0,,,
6,4295906579,1992-12-31T00:00:00Z,40323.688,32591.1744,,,Genelabs Technologies Inc
7,4295906579,1993-12-31T00:00:00Z,116225.184,68018.2496,,,
8,4295906579,1994-12-31T00:00:00Z,130984.2496,84303.1721,,,
9,4295906579,1995-12-31T00:00:00Z,146030.6232,98340.4422,,,


### symbology converter

In [ ]:
firms = ['GOOG.O', 'MSFT.O', 'FB.O', 'AMZN.O', 'TWTR.K'] # check the screener
fields = ['TR.RIC', 'TR.CUSIP', 'TR.SEDOL', 'TR.ISIN', 'TR.OrganizationID'] # check the data item browser
params = {'Scale': 6, 'SDate': 0, 'EDate': 0, 'FRQ': 'FY', 'Curn': 'USD'} # set EDate to minus values for the most recent years

df, err = ek.get_data(firms, fields, params)

df

,Instrument,RIC,CUSIP,SEDOL,ISIN,Organization PermID
0,GOOG.O,GOOG.O,02079K107,BYY88Y7,US02079K1079,5030853586
1,MSFT.O,MSFT.O,594918104,2588173,US5949181045,4295907168
2,FB.O,FB.O,30303M102,B7TL820,US30303M1027,4297297477
3,AMZN.O,AMZN.O,023135106,2000019,US0231351067,4295905494
4,TWTR.K,TWTR.K,90184L102,BFLR866,US90184L1026,4296301199


In [ ]:
firm_ids = df["RIC"].tolist()
df_symb = ek.get_symbology(firm_ids, from_symbol_type='RIC', to_symbol_type=None, raw_output=False, debug=False)
# to_symbol_type=None -> all are requested. CUSIP , ISIN , SEDOL , RIC , ticker , lipperID , IMO , and OAPermID

df_symb

,CUSIP,ISIN,OAPermID,RIC,ticker
GOOG.O,02079K107,US02079K1079,5030853586,GOOG.O,GOOG
MSFT.O,594918104,US5949181045,4295907168,MSFT.O,MSFT
FB.O,30303M102,US30303M1027,4297297477,META.O,META
AMZN.O,023135106,US0231351067,4295905494,AMZN.O,AMZN
TWTR.K,90184L102,US90184L1026,4296301199,TWTR.K,TWTR
